# Experiment with recognizing times from MK8DX screenshots

In [1]:
from transformers import AutoModel, AutoTokenizer

In [2]:
tokenizer = AutoTokenizer.from_pretrained('ucaslcl/GOT-OCR2_0', trust_remote_code=True)

c:\Users\steve\OneDrive\Documents\GitHub\mk8dx-lap-time-tracker\.venv\lib\site-packages\huggingface_hub\file_download.py:795: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [3]:
model = AutoModel.from_pretrained('ucaslcl/GOT-OCR2_0', trust_remote_code=True, low_cpu_mem_usage=True, device_map='cuda', use_safetensors=True, pad_token_id=tokenizer.eos_token_id)

c:\Users\steve\OneDrive\Documents\GitHub\mk8dx-lap-time-tracker\.venv\lib\site-packages\huggingface_hub\file_download.py:795: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


AssertionError: Torch not compiled with CUDA enabled

In [8]:
model = model.eval().cuda()

AssertionError: Torch not compiled with CUDA enabled

In [6]:
image_file = 'data/test_images/2019102710534900_c.jpg'

In [7]:
res = model.chat(tokenizer, image_file, ocr_type='ocr')

AssertionError: Torch not compiled with CUDA enabled